# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install ..

Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.
Processing /Users/bytedance/Developer/todd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for todd-ai: filename=todd_ai-0.4.0-py3-none-any.whl size=106529 sha256=36400bf999963bc8d235c5aac6befc5818d4cc6a096686754d6cd8b11016a0b7
  Stored in directory: /private/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/pip-ephem-wheel-cache-a__q8cpm/wheels/15/ef/5a/9fc12e257ce5cef16b333a2ed6c992ff9cbcc9167f7199e6ac
Successfully built todd-ai

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pathlib
import tempfile
from pprint import pprint
from typing import Any, NoReturn, TypedDict, cast

import todd
import torch
import torch.nn.functional as F
import torch.utils.data

Memo = dict[str, Any]

/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
[2023-08-02 23:27:34,023 74460:140704362395200][patches.py:14 todd <module>] INFO: `ipdb` is installed. Using it for debugging.


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register()
class RunnerModel(todd.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

### Runners

In [7]:
class RunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        y: torch.Tensor = self._strategy.model(batch['x'])
        loss = F.l1_loss(y, batch['y'])
        memo['loss'] = loss
        if 'log' in memo:
            memo['log']['loss'] = f'{loss.item():.3f}'
        return memo

In [8]:
class TrainerMixin(RunnerMixin):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        memo = super()._run_iter(batch, memo)
        if 'log' in memo:
            model = cast(RunnerModel, self._strategy.module)
            memo['log']['weight'] = f'{model.weight.item():.3f}'
            memo['log']['batch'] = str(batch)
        return memo

In [9]:
@todd.RunnerRegistry.register()
class CustomValidator(RunnerMixin, todd.runners.Validator):
    pass

In [10]:
@todd.RunnerRegistry.register()
class CustomIterBasedTrainer(TrainerMixin, todd.runners.IterBasedTrainer):
    pass

In [11]:
@todd.RunnerRegistry.register()
class CustomEpochBasedTrainer(TrainerMixin, todd.runners.EpochBasedTrainer):
    pass

## Validators

In [12]:
validator_demo = todd.Config(
    type='CustomValidator',
    name='custom_validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
)

In [13]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-08-02 23:27:34,856 74460:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpu456jlu7
└── custom_validator

2 directories, 0 files


In [14]:
validator_demo.callbacks=dict(type='LogCallback', interval=5)

In [15]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-08-02 23:27:35,219 74460:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:35,227 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.010833 loss=10.000
[2023-08-02 23:27:35,231 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:00:00.007368 loss=20.000
[2023-08-02 23:27:35,233 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] ETA 0:00:00.003283 loss=30.000
[2023-08-02 23:27:35,235 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] ETA 0:00:00 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpj5tb9eze
└── custom_validator

2 directories, 0 files


## Trainers

In [16]:
trainer_demo = validator_demo.copy()
trainer_demo.pop('type')
trainer_demo.dataloader = todd.Config(
    batch_size=2,
    shuffle=True,
    dataset=dict(type='RunnerDataset', n=67),
)
trainer_demo.optimizer = todd.Config(type='SGD', lr=0.005)


### Iteration Based

In [17]:
iter_based_trainer_demo = trainer_demo.copy()
iter_based_trainer_demo.type = 'CustomIterBasedTrainer'
iter_based_trainer_demo.name = 'custom_iter_based_trainer'
iter_based_trainer_demo.iters = 53

In [18]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        iter_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 23:27:35,545 74460:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:35,550 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.030058 loss=50.000 weight=0.000 batch={'x': tensor([24, 26]), 'y': tensor([48, 52])}
[2023-08-02 23:27:35,553 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.024691 loss=70.000 weight=0.000 batch={'x': tensor([64,  6]), 'y': tensor([128,  12])}
[2023-08-02 23:27:35,556 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] ETA 0:00:00.021822 loss=29.000 weight=0.000 batch={'x': tensor([15, 14]), 'y': tensor([30, 28])}
[2023-08-02 23:27:35,558 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom

### Epoch Based

In [19]:
epoch_based_trainer_demo = trainer_demo.copy()
epoch_based_trainer_demo.type = 'CustomEpochBasedTrainer'
epoch_based_trainer_demo.name = 'custom_epoch_based_trainer'
epoch_based_trainer_demo.epochs = 3

In [20]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        epoch_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 23:27:35,594 74460:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:35,596 74460:140704362395200][log.py:87 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 23:27:35,599 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.067628 loss=68.000 weight=0.000 batch={'x': tensor([58, 10]), 'y': tensor([116,  20])}
[2023-08-02 23:27:35,602 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] ETA 0:00:00.056801 loss=62.000 weight=0.000 batch={'x': tensor([41, 21]), 'y': tensor([82, 42])}
[2023-08-02 23:27:35,604 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] ETA 0:00:00.049097 loss=69.0

## Callbacks

### Log

In [21]:
log_callback_demo = validator_demo.copy()
log_callback = log_callback_demo.callbacks
log_callback.collect_env = todd.Config(verbose=False)
log_callback.with_file_handler = True
log_callback_demo.callbacks = [log_callback]

In [22]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        log_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-08-02 23:27:35,739 74460:140704362395200][log.py:48 todd.CustomValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:35,742 74460:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:35,746 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.006705 loss=10.000
[2023-08-02 23:27:35,748 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:00:00.004505 loss=20.000
[2023-08-02 23:27:35,751 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/2


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp9l81by23
└── custom_validator
    └── 2023-08-02T23-27-35_674616-08-00.log

2 directories, 1 file

[2023-08-02 23:27:35,739 74460:140704362395200][log.py:48 todd.CustomValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:35,742 74460:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:35,746 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.006705 loss=10.000
[2023-08-02 23:27:35,748 74460:140704362395200][log.py:81 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:0

### Optimize

In [23]:
optimize_callback_demo = iter_based_trainer_demo.copy()
optimize_callback = todd.Config(type='OptimizeCallback')
optimize_callback_demo.callbacks = [optimize_callback, log_callback]

In [24]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        optimize_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 23:27:36,434 74460:140704362395200][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:36,437 74460:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:36,446 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.068026 loss=23.760 weight=0.515 batch={'x': tensor([11, 21]), 'y': tensor([22, 42])}
[2023-08-02 23:27:36,452 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.053625 loss=40.32

### Learning Rate Schedule

In [25]:
lr_schedule_callback_demo = iter_based_trainer_demo.copy()
lr_schedule_callback = todd.Config(
    type='LRScheduleCallback',
    lr_scheduler=dict(type='LinearLR', total_iters=10),
)
lr_schedule_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_callback,
    log_callback,
]


In [26]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 23:27:36,587 74460:140704362395200][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:36,591 74460:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:36,602 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.082128 loss=40.269 weight=0.249 batch={'x': tensor([45,  1]), 'y': tensor([90,  2])} lr=['3.333e-03']
[2023-08-02 23:27:36,612 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.

In [27]:
lr_schedule_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
lr_schedule_by_epoch_callback = lr_schedule_callback.copy()
lr_schedule_by_epoch_callback.by_epoch = True
lr_schedule_by_epoch_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_by_epoch_callback,
    log_callback,
]


In [28]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_by_epoch_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 23:27:36,750 74460:140704362395200][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:36,753 74460:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:36,755 74460:140704362395200][log.py:87 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 23:27:36,762 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.141368 loss=60.777 weight=0.238 batch={'x': tensor([60,  9]), 'y': tensor([120,  18])} lr=['1.667e-03']


### Learning Rate Scaler

In [29]:
lr_scaler_callback_demo = iter_based_trainer_demo.copy()
lr_scaler_callback = todd.Config(
    type='LRScaleCallback',
    lr_scaler=dict(base_batch_size=1),
)
lr_scaler_callback_demo.callbacks = [
    optimize_callback,
    lr_scaler_callback,
    log_callback,
]

In [30]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_scaler_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 23:27:36,954 74460:140704362395200][lr.py:92 todd.CustomIterBasedTrainer.custom_iter_based_trainer _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2023-08-02 23:27:37,076 74460:140704362395200][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:37,079 74460:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:37,090 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.052666 loss=27.405 weight=1.370 batch={'x': tensor([59, 28]), 'y': tensor([118,  56])}

### Checkpoint

In [31]:
checkpoint_callback_demo = iter_based_trainer_demo.copy()
checkpoint_callback = todd.Config(type='CheckpointCallback', interval=10)
checkpoint_callback_demo.callbacks = [checkpoint_callback, log_callback]

In [32]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_50 = pathlib.Path(work_dirs) / 'custom_iter_based_trainer' / 'checkpoints' / 'iter_50'
    for f in iter_50.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-08-02 23:27:37,255 74460:140704362395200][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:37,259 74460:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:37,267 74460:140704362395200][log.py:81 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.054778 loss=65.000 weight=0.000 batch={'x': tensor([45, 20]), 'y': tensor([90, 40])}
[2023-08-02 23:27:37,272 74460:140704362395200][checkpoint.py:59 todd.CustomIterBasedTrainer.custom_iter_based_trainer _save] INFO: Saving state dict to /var/folders/v_/1kkfntx

/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpzmzmo5z7
└── custom_iter_based_trainer
    ├── 2023-08-02T23-27-37_195868-08-00.log
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_20
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_30
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_40
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_50
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├──

In [33]:
checkpoint_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
checkpoint_by_epoch_callback = checkpoint_callback.copy()
checkpoint_by_epoch_callback.update(interval=1, by_epoch=True)
checkpoint_by_epoch_callback_demo.callbacks = [
    checkpoint_by_epoch_callback,
    log_callback,
]

In [34]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_by_epoch_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'custom_epoch_based_trainer' / 'checkpoints' / 'epoch_2'
    for f in epoch_2.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-08-02 23:27:37,907 74460:140704362395200][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:37,909 74460:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:37,912 74460:140704362395200][log.py:87 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 23:27:37,918 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.114014 loss=35.000 weight=0.000 batch={'x': tensor([ 5, 30]), 'y': tensor([10, 60])}
[2023-08-02 23:27:3


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpq5ob6p67
└── custom_epoch_based_trainer
    ├── 2023-08-02T23-27-37_824195-08-00.log
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├── meta.pth
            ├── model.pth
            ├── optim.pth
            └── strategy.pth

7 directories, 21 files

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                 

In [35]:
checkpoint_load_from_callback_demo = checkpoint_by_epoch_callback_demo.copy()
checkpoint_load_from_callback_demo.callbacks = [
    optimize_callback,
    checkpoint_by_epoch_callback,
    log_callback,
]

In [36]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
        load_from=os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2')
    )
    runner.run()

[2023-08-02 23:27:38,613 74460:140704362395200][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:38,616 74460:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:38,618 74460:140704362395200][log.py:87 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 23:27:38,626 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.157916 loss=14.035 weight=0.997 batch={'x': tensor([ 5, 23]), 'y': tensor([10, 46])}
[2023-08-02 23:27:3


--------------------



[2023-08-02 23:27:39,268 74460:140704362395200][checkpoint.py:44 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpounurhw4/custom_epoch_based_trainer/checkpoints/epoch_2
[2023-08-02 23:27:39,333 74460:140704362395200][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:39,335 74460:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:39,338 74460:140704362395200][log.py:87 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [3/3]
[2023-08-02

### Monitor

In [37]:
class CustomError(RuntimeError):
    pass

In [38]:
class FaultyRunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError('faulty runner')

In [39]:
@todd.RunnerRegistry.register()
class FaultyValidator(FaultyRunnerMixin, todd.runners.Validator):
    pass

In [40]:
@todd.RunnerRegistry.register()
class FaultyIterBasedTrainer(FaultyRunnerMixin, todd.runners.IterBasedTrainer):
    pass

In [41]:
@todd.RunnerRegistry.register()
class FaultyEpochBasedTrainer(
    FaultyRunnerMixin,
    todd.runners.EpochBasedTrainer,
):
    pass

In [42]:
monitor_callback_demo = validator_demo.copy()
monitor_callback_demo.type = 'FaultyValidator'
monitor_callback = todd.Config(type='MonitorCallback')
monitor_callback_demo.callbacks = [monitor_callback, log_callback]

In [43]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        monitor_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-08-02 23:27:39,564 74460:140704362395200][log.py:48 todd.FaultyValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:39,566 74460:140704362395200][base.py:53 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:39,571 74460:140704362395200][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x157feba10>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 193, in _run
    memo = self.

[2023-08-02 23:27:39,564 74460:140704362395200][log.py:48 todd.FaultyValidator.custom_validator init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:39,566 74460:140704362395200][base.py:53 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:39,571 74460:140704362395200][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x157feba10>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 193, in _run
    memo = self.

### Priorities

## Strategies

In [44]:
strategy_load_model_from_demo = checkpoint_load_from_callback_demo.copy()

In [45]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.load_model_from(os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2', 'model.pth'))
    runner.run()

[2023-08-02 23:27:40,018 74460:140704362395200][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:40,021 74460:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:40,023 74460:140704362395200][log.py:87 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 23:27:40,030 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.133763 loss=26.048 weight=0.867 batch={'x': tensor([41,  5]), 'y': tensor([82, 10])}
[2023-08-02 23:27:4

[2023-08-02 23:27:40,131 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [90/102] ETA 0:00:00.014355 loss=8.092 weight=2.195 batch={'x': tensor([64, 19]), 'y': tensor([128,  38])}
[2023-08-02 23:27:40,136 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [95/102] ETA 0:00:00.008334 loss=0.975 weight=2.162 batch={'x': tensor([8, 4]), 'y': tensor([16,  8])}
[2023-08-02 23:27:40,141 74460:140704362395200][log.py:81 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [100/102] ETA 0:00:00.002348 loss=11.155 weight=2.230 batch={'x': tensor([48, 49]), 'y': tensor([96, 98])}
[2023-08-02 23:27:40,143 74460:140704362395200][checkpoint.py:59 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpx1fho5w5/custom_epoch_based_trainer/checkpoints/epoch


--------------------



[2023-08-02 23:27:40,622 74460:140704362395200][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: 
Platform: macOS-13.4.1-x86_64-i386-64bit
NVIDIA SMI: None
Python version: 3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]
PyTorch version: 2.0.1
TorchVision version: 0.15.2
OpenCV version: 4.7.0
Todd version: 0.4.0
CUDA_HOME: None
Git commit ID: 093521c
[2023-08-02 23:27:40,625 74460:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 23:27:40,628 74460:140704362395200][base.py:64 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpx1fho5w5/custom_epoch_based_trainer/checkpoints/epoch_2/model.pth
[2023-08-02 23:27:40,632 74460:140704362395200][log.py:87 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Ep

## Dry Run

In [46]:
todd.Store.DRY_RUN = True